In [1]:
import requests
import json
import pandas as pd


In [2]:
url ="http://publicplansdata.org/api/"

In [3]:
params = {
    "q": "ListDataSets",
     "format": "json"
    
}


In [4]:
response = requests.get(url, params = params)
print(response.url)

http://publicplansdata.org/api/?q=ListDataSets&format=json


In [5]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-27 03:27:34',
  'q': 'listdatasets',
  'params': 'q=ListDataSets&format=json',
  'recordcount': 34},
 {'id': '3',
  'table_name': 'pensiongasbschedules',
  'table_display_name': 'Funding'},
 {'id': '4',
  'table_name': 'pensiongasbassumptions',
  'table_display_name': 'Assumptions and Methods'},
 {'id': '5',
  'table_name': 'pensionincomestatement',
  'table_display_name': 'Income Statement'},
 {'id': '7',
  'table_name': 'pensioninvestmentreturn',
  'table_display_name': 'Investment Returns'},
 {'id': '8',
  'table_name': 'pensionactuarialcosts',
  'table_display_name': 'Actuarial Costs'},
 {'id': '9',
  'table_name': 'pensionprovisions',
  'table_display_name': 'Plan Provisions'},
 {'id': '10',
  'table_name': 'pensionmembership',
  'table_display_name': 'Membership'},
 {'id': '11',
  'table_name': 'pensionreportingdates',
  'table_display_name': 'Plan Reporting'},
 {'id': '12',
  'table_name': 'pensioneegroupbasics',
  'table_display_name': 'Empl

In [6]:
params2 = {
    "q": "pensionsystemdata",
     "format": "json",
    #"variables": 
     #"v": variables,
}


In [7]:
response = requests.get(url, params = params2)
print(response.url)

http://publicplansdata.org/api/?q=pensionsystemdata&format=json


In [8]:
data = response.json()
data

[{'status': 'ERROR',
  'date': '2021-05-27 03:27:34',
  'q': 'pensionsystemdata',
  'params': 'q=pensionsystemdata&format=json',
  'recordcount': 1},
 {'Error': "Unknown query type 'pensionsystemdata'"}]

In [9]:
params3 = {
    "q": "QDataSet",
    "dataset": "pensiongasbschedules",
     "format": "json",
    #"variables": 
     #"v": variables,
}


In [10]:
response = requests.get(url, params = params3)
print(response.url)

http://publicplansdata.org/api/?q=QDataSet&dataset=pensiongasbschedules&format=json


In [11]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-27 03:27:34',
  'q': 'qdataset',
  'params': 'q=QDataSet&dataset=pensiongasbschedules&format=json',
  'recordcount': 9815},
 {'ActAssets_est': '0.000',
  'ActAssets_GASB': '8028471.000',
  'ActFundedRatio_est': '0.000',
  'ActFundedRatio_GASB': '1.00200',
  'ActFundedRatio_GASB67': None,
  'ActLiabilities_est': '0.000',
  'ActLiabilities_GASB': '8010123.000',
  'ActLiabilities_other': '0.000',
  'ADEC': None,
  'EEGroupID': '0',
  'fy': '2001',
  'NetPensionLiability': None,
  'NetPosition': None,
  'notes_GASBSchedules': None,
  'payroll': '2408543.000',
  'PercentReqContPaid': '1.00000',
  'ppd_id': '1',
  'RequiredContribution': '122483.000',
  'RequiredContribution_est': None,
  'source_GASBSchedules': None,
  'TierID': '0',
  'TotalPensionLiability': None,
  'UAAL_GASB': '-18348.000'},
 {'ActAssets_est': '0.000',
  'ActAssets_GASB': '8100846.000',
  'ActFundedRatio_est': '0.000',
  'ActFundedRatio_GASB': '0.95400',
  'ActFundedRatio_GASB67': No

In [12]:
data_df = pd.DataFrame(data[1:])


In [13]:
data_df.iloc[0:10,0:5]

,ActAssets_est,ActAssets_GASB,ActFundedRatio_est,ActFundedRatio_GASB,ActFundedRatio_GASB67
0,0.000,8028471.000,0.000,1.00200,None
1,0.000,8100846.000,0.000,0.95400,None
2,0.000,8312500.000,0.000,0.91100,None
3,0.000,8563945.000,0.000,0.89700,None
4,0.000,8935358.000,0.000,0.84000,None
5,0.000,9287531.000,0.000,0.81100,None
6,0.000,9770897.000,0.000,0.79000,None
7,0.000,9905766.000,0.000,0.75700,None
8,0.000,9928104.000,0.000,0.72200,None
9,0.000,9739331.000,0.000,0.68200,None


In [14]:
data_df.head()

,ActAssets_est,ActAssets_GASB,ActFundedRatio_est,ActFundedRatio_GASB,ActFundedRatio_GASB67,ActLiabilities_est,ActLiabilities_GASB,ActLiabilities_other,ADEC,EEGroupID,...,notes_GASBSchedules,payroll,PercentReqContPaid,ppd_id,RequiredContribution,RequiredContribution_est,source_GASBSchedules,TierID,TotalPensionLiability,UAAL_GASB
0,0.000,8028471.000,0.000,1.00200,None,0.000,8010123.000,0.000,None,0,...,None,2408543.000,1.00000,1,122483.000,None,None,0,None,-18348.000
1,0.000,8100846.000,0.000,0.95400,None,0.000,8493469.000,0.000,None,0,...,None,2547775.000,1.00000,1,123887.000,None,None,0,None,392623.000
2,0.000,8312500.000,0.000,0.91100,None,0.000,9124279.000,0.000,None,0,...,None,2677025.000,1.00000,1,154218.000,None,None,0,None,811779.000
3,0.000,8563945.000,0.000,0.89700,None,0.000,9546478.000,0.000,None,0,...,None,2702393.000,1.00000,1,170713.000,None,None,0,None,982533.000
4,0.000,8935358.000,0.000,0.84000,None,0.000,10634976.000,0.000,None,0,...,None,2982122.000,1.00000,1,195846.000,None,None,0,None,1699618.000
